In [1]:
import numpy as np
from scipy.stats import binom
from scipy.stats import truncnorm

In [2]:
# Parameters 

h = 1/7
pho = 2/3
gamma = 2/3
R = 19
T = 7*R

In [3]:
def truncated_normal(mean=0.5, sd=0.5, low=0, upp=1):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

In [4]:
#  Effective Sampling Size 

def ESS(W):                 # Takes an array as argument
    return 1/np.sum(W**2)

In [5]:
# Mutation transition

def mutation(theta, X):
    #
    X_new = X
    S, E, I, R =  X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3]
    M = theta[:16].reshape(4,4)
    betas = 1/8000 * np.dot(M, I)
    B = np.random.binomial(S, np.ones(4)-np.exp(-h*betas)) 
    C = np.random.binomial(E, 1-np.exp(-h*pho)) 
    D = np.random.binomial(I, 1-np.exp(-h*gamma)) 
    S_new = S - B
    E_new = E + B - C
    I_new = I + C - D
    R_new = R + D
    X_new[:, 0], X_new[:, 1], X_new[:, 2], X_new[:, 3] = S_new, E_new, I_new, R_new
    return X_new

In [6]:
# Likelihood of y knowing theta

def log_likelihood(y, theta, X):
    C = np.zeros((7,4))
    for i in range(7):
        X = mutation(theta, X)
        C[i] = np.random.binomial(X[:, 1], 1-np.exp(-h*pho))
    return(np.log(binom.pmf(y, np.sum(C, axis=0), theta[16:])+1.5))

In [25]:
# Initial settings
X1 = np.array([948, 0, 1, 0])
X2 = np.array([1689, 0, 1, 0])
X3 = np.array([3466, 0, 1, 0])
X4 = np.array([1894, 0, 1, 0])

def BootstrapFilter(y, theta, ESS_min, N, R=19):
    X = np.array([[X1, X2, X3, X4] for i in range(N)]) 
    w_init = np.zeros((N, 4))
    X_all = np.array([X for r in range(R+1)])
    #print(X_all)
    for n in range(N):
        w_init[n] = log_likelihood(y[0], theta, X[n])
    W_init = np.zeros((N, 4))
    for n in range(N):
        W_init[n] = np.divide(w_init[n], np.sum(w_init, axis=0))
    W = W_init
    w = w_init
    X_all[1] = X
    #print(X_all)
    for r in range(R):
        #print(X)
        w_hats = np.zeros((N,4))
        #print(w)
        for k in range(4):
            #print(k, W[:, k], ESS(W[:, k]))
            if ESS(W[:, k]) < ESS_min:
                A = np.random.choice(np.arange(0, N), size=N, p=W[:, k])
                w_hat = np.zeros(N)
            else:
                A = np.arange(0, N)
                w_hat = w[:, k]
            for n in range(N):
                X[n][k] = X[A[n]][k]
            w_hats[:, k] = w_hat
        for n in range(N):
            w[n] = w_hats[n]+log_likelihood(y[r], theta, X[n])
        for n in range(N):
            W[n] = np.divide(w[n], np.sum(w, axis=0))
        X_all[r+1] = X
    return(X_all)

In [31]:
theta = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
Y = np.array([[0,0,1,1],[0,2,6,1],[0,2,4,2],[23,73,63,11],[63,208,173,41],[73,207,171,27],[66,150,143,7],[26,40,87,29],[17,18,33,12],[3,4,13,6],[2,6,16,5],[1,6,11,3],[0,1,6,5],[0,2,2,2],[0,1,3,0],[0,1,4,6],[0,1,3,0],[2,1,7,1], [1,1,6,2]])
X_all = BootstrapFilter(Y, theta, , 100, R=19)

TypeError: 'float' object cannot be interpreted as an integer

In [30]:
print(X_all[8])

[[[ 101  226  250  372]
  [  44  297  446  903]
  [   2  353  729 2383]
  [  45  343  484 1023]]

 [[ 110  243  242  354]
  [  41  328  452  869]
  [  13  602  833 2019]
  [  48  339  514  994]]

 [[ 117  251  230  351]
  [  32  310  461  887]
  [   4  375  717 2371]
  [  37  366  495  997]]

 ...

 [[ 111  240  221  377]
  [  26  320  441  903]
  [   8  432  761 2266]
  [  44  362  519  970]]

 [[ 114  226  248  361]
  [  38  323  423  906]
  [   6  440  848 2173]
  [  46  355  514  980]]

 [[  95  233  226  395]
  [  34  299  451  906]
  [   7  415  806 2239]
  [  43  324  513 1015]]]


In [ ]:
def LogL(y, theta, X):
    L = 0
    for r in range(R):
        S = 0
        for n in range(len(X[r]):
            x = X[r]
            S = S + log_likelihood(y[r], theta, x)

In [504]:
def PMMH(y, ESS_min, Nx, Nt, R=19):
    thetas = []
    theta = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
    thetas.append(theta)
    X = BootstrapFilter(y, theta, ESS_min, Nx, R=19)
    L = LogL(y, theta, X)
    for n in range(Nt):
        theta_new = np.concatenate((np.random.gamma(5, 1, 16), truncated_normal(mean=0.5, sd=0.5, low=0, upp=1).rvs(4)), axis=None)
        X_new = BootstrapFilter(y, theta_new, ESS_min, Nx, R=19)
        L_new = LogL(y, theta_new, X_new)
        v = L_new - L
        u = np.random.uniform()
        if np.log(u)<v:
            L = L_new
            thetas.append(theta_new)
        else:
            thetas.append(thetas[-1])
    return thetas